# 50.007 Machine Learning Project 2023
- Beckham Wee 1006010
- Shao Ren 1005935
- Zheng Wei 1006014

# Part 1

Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation):

In [1]:
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_content = file.read()
    return file_content

def write_file(file_path, content):
    with open(file_path, 'w', encoding="utf-8") as file:
        file.write(content)

In [2]:
def estimate_emission_parameters(file_content):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in lines:
        if len(line) == 0:
            continue
        if line[-1] == 'O':
            observation, tag = line[:-1], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-10], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-9], line[-9:]
#         observation, tag = line.split()
    
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: count / total_count for observation, count in observations_count.items()
        }

    return emission_params

file_content = read_file("ES/train.txt")

emission_parameters = estimate_emission_parameters(file_content)
# print(emission_parameters, sum(len(words) for words in emission_parameters.values()))

One problem with estimating the emission parameters is that some words that appear in the test set
do not appear in the training set. One simple idea to handle this issue is as follows. We introduce
a special word token #UNK#, and make the following modifications to the computation of emission
probabilities.During the testing phase, if the word does not appear in the training set, we replace that word with
#UNK#. Set k to 1, implement this fix into your function for computing the emission parameters.

In [3]:
def estimate_emission_parameters_modified(training_file_content, test_file_content, k=1):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities
    train_words = []
    test_words = []
    
    # Split the file content into lines
    train_data_lines = training_file_content.strip().split("\n")
    test_data_lines = test_file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in train_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        if observation not in train_words:
            train_words.append(observation)
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}
    # Iterate through each line to extract observations from test set
    for line in test_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        else:
            observation = line
        if observation not in test_words:
            test_words.append(observation)
    # Extract words that are in test set but not in train set   
    unique_words = find_unique_words(test_words, train_words)

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: count / (total_count + k) for observation, count in observations_count.items()
        }
        for word in unique_words:
            emission_params[tag][word] = k / (total_count + k)


    return emission_params, list(set(test_words).union(train_words))

def find_unique_words(list1, list2):
    set1 = set(list1)
    set2 = set(list2)

    unique_in_list1 = set1 - set2

    return list(unique_in_list1)


training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_parameters = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
# print(emission_parameters, sum(len(words) for words in emission_parameters.values()))
count = 0
for emission_probabilities in emission_parameters.values():
    count += len(emission_probabilities)
# print(count)

In [4]:
def sentiment_analysis(path_to_train_file, path_to_test_file):
    training_file_content = read_file(path_to_train_file)
    test_file_content = read_file(path_to_test_file)
    emission_parameters, words = estimate_emission_parameters_modified(training_file_content, test_file_content)
    word_to_label = {}
    for word in words:
        probabilities = []
        for label, freqs in emission_parameters.items():
            if word in freqs:
                probabilities.append((label, freqs[word]))
        word_to_label[word] = max(probabilities, key=lambda x: x[1])[0]
    return word_to_label

def write_result_to_file(word_to_label, path_to_dev_set, path_to_output):
    result = ""
    dev_file_content = read_file(path_to_dev_set)
    dev_set_lines = dev_file_content.strip().split("\n")
    counter = 0
    for line in dev_set_lines:
        if line == "":
            result += "\n"
            continue  
        result += line + " " + word_to_label.get(line) + "\n"
 
    write_file(path_to_output, result)        

In [5]:
mapping = sentiment_analysis("ES/train.txt", "ES/dev.in")
write_result_to_file(mapping, "ES/dev.in", "ES/dev.p1.out")

In [6]:
mapping = sentiment_analysis("RU/train.txt", "RU/dev.in")
write_result_to_file(mapping, "RU/dev.in", "RU/dev.p1.out")

## ES
#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145

## RU
#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170

# Part 2

Write a function that estimates the transition parameters from the training set using MLE (maximum
likelihood estimation):

In [7]:
def estimate_transition_parameters(file_content):
    transition_counts = {}
    transition_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line_index in range(len(lines)):
        next_line = ""
        curr_line = lines[line_index]
        if line_index < len(lines) - 1:
            next_line = lines[line_index + 1]
        
        if len(curr_line) == 0:
            continue
        if curr_line[-1] == 'O':
            curr_observation, curr_tag = curr_line[:-1], curr_line[-1]
        elif ('B-positive' in curr_line 
            or 'I-positive' in curr_line 
            or 'B-negative' in curr_line 
            or 'I-negative' in curr_line):
            curr_observation, curr_tag = curr_line[:-10], curr_line[-10:]
        elif ('B-neutral' in curr_line
            or 'I-neutral' in curr_line):
            curr_observation, curr_tag = curr_line[:-9], curr_line[-9:]
        
        if len(next_line) == 0: #true if line_index = len(lines - 1) or line_index is last index of a document
            pass
        elif next_line[-1] == 'O':
            next_observation, next_tag = next_line[:-1], next_line[-1]
        elif ('B-positive' in next_line 
            or 'I-positive' in next_line 
            or 'B-negative' in next_line 
            or 'I-negative' in next_line):
            next_observation, next_tag = next_line[:-10], next_line[-10:]
        elif ('B-neutral' in next_line
            or 'I-neutral' in next_line):
            next_observation, next_tag = next_line[:-9], next_line[-9:]        
        
        # Handle transitions from START
        if line_index == 0 or lines[line_index - 1] == "":
            if 'START' in transition_counts:
                transition_counts['START'][curr_tag] = transition_counts['START'].get(curr_tag, 0) + 1
            else:
                transition_counts['START'] = {curr_tag: 1}
                
        # Handle transitions to STOP
        if len(next_line) == 0:
            if curr_tag in transition_counts:
                transition_counts[curr_tag]['STOP'] = transition_counts[curr_tag].get('STOP', 0) + 1
            else:
                transition_counts[curr_tag] = {'STOP': 1}
                
        # Rest of the transitions
        else:
            if curr_tag in transition_counts:
                transition_counts[curr_tag][next_tag] = transition_counts[curr_tag].get(next_tag, 0) + 1
            else:
                transition_counts[curr_tag] = {next_tag: 1}
        

    # Calculate transition probabilities 
    for tag, next_tag_counts in transition_counts.items():
        total_count = sum(next_tag_counts.values())
        transition_params[tag] = {
            next_tag: count / total_count for next_tag, count in next_tag_counts.items()
        }

    return transition_params

file_content = read_file("ES/train.txt")

transition_parameters = estimate_transition_parameters(file_content)
print(transition_parameters)

{'START': {'O': 0.9289176090468497, 'B-positive': 0.052234787291330104, 'B-negative': 0.014001077005923533, 'B-neutral': 0.004846526655896607}, 'O': {'O': 0.8856896848630963, 'B-positive': 0.03650766316514551, 'STOP': 0.06344067504735663, 'B-negative': 0.012226623041157224, 'B-neutral': 0.0021353538832443605}, 'B-positive': {'O': 0.871551724137931, 'I-positive': 0.11637931034482758, 'STOP': 0.008620689655172414, 'B-neutral': 0.0008620689655172414, 'B-positive': 0.002586206896551724}, 'B-negative': {'O': 0.8110236220472441, 'I-negative': 0.1784776902887139, 'STOP': 0.010498687664041995}, 'B-neutral': {'I-neutral': 0.20833333333333334, 'O': 0.7916666666666666}, 'I-neutral': {'I-neutral': 0.6511627906976745, 'O': 0.3488372093023256}, 'I-positive': {'I-positive': 0.5700636942675159, 'O': 0.4267515923566879, 'STOP': 0.0031847133757961785}, 'I-negative': {'O': 0.39766081871345027, 'I-negative': 0.6023391812865497}}


In [8]:
def viterbi(observation_sequence, emission_params, transition_params):
    states = list(emission_params.keys())
    T = len(observation_sequence)
    N = len(states)

    # Initialization step
    viterbi_table = [{}]
    backpointer = [{}]
    for state in states:
        viterbi_table[0][state] = transition_params['START'].get(state, 0) * emission_params[state].get(observation_sequence[0], 0)
        backpointer[0][state] = None

    # Recursion step
    for t in range(1, T):
        viterbi_table.append({})
        backpointer.append({})
        for state in states:
            max_prob = max(
                viterbi_table[t - 1][prev_state] * transition_params[prev_state].get(state, 0) * emission_params[state].get(observation_sequence[t], 0)
                for prev_state in states
            )
            viterbi_table[t][state] = max_prob
            backpointer[t][state] = max(states, key=lambda prev_state: viterbi_table[t - 1][prev_state] * transition_params[prev_state].get(state, 0))

    # Termination step
    max_prob_last_state = max(viterbi_table[T - 1].values())
    best_last_state = max(states, key=lambda state: viterbi_table[T - 1][state] * transition_params[state].get('STOP', 0))

    # Backtrack to find the best sequence
    best_sequence = [best_last_state]
    prev_state = best_last_state
    for t in range(T - 2, -1, -1):
        best_sequence.insert(0, backpointer[t + 1][prev_state])
        prev_state = backpointer[t + 1][prev_state]

    return best_sequence

def create_observation_sequences(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_content = file.read().strip()

    observation_sequences = []
    current_sequence = []

    # Split the file content by empty lines to get individual sequences
    sequence_lines = file_content.split("\n\n")

    for sequence_line in sequence_lines:
        # Split each sequence into individual observations (words)
        observation_sequence = sequence_line.strip().split("\n")
        current_sequence.extend(observation_sequence)
        observation_sequences.append(current_sequence)
        current_sequence = []

    return observation_sequences

def viterbi_for_sequences(observation_sequences, emission_params, transition_params):
    best_sequences = []
    for observation_sequence in observation_sequences:
        best_sequence = viterbi(observation_sequence, emission_params, transition_params)
        best_sequences.append(best_sequence)

    return best_sequences

def write_sequences_to_file(output_file_path, observation_sequences, best_sequences):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for obs_sequence, best_sequence in zip(observation_sequences, best_sequences):
            for word, tag in zip(obs_sequence, best_sequence):
                file.write(f"{word} {tag}\n")
            file.write("\n")  # Separate sequences with an empty line

In [9]:
training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("ES/dev.p2.out", observation_sequences, best_sequences)

In [10]:
training_file_content = read_file("RU/train.txt")
test_file_content = read_file("RU/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("RU/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("RU/dev.p2.out", observation_sequences, best_sequences)

## ES
#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516

## RU

#Entity in gold data: 389
#Entity in prediction: 514

#Correct Entity : 190
Entity  precision: 0.3696
Entity  recall: 0.4884
Entity  F: 0.4208

#Correct Sentiment : 129
Sentiment  precision: 0.2510
Sentiment  recall: 0.3316
Sentiment  F: 0.2857

# Part 3
Use the estimated transition and emission parameters, implement an algorithm to find the k-th best
output sequences (if there are multiple sequences that are k-th best, output any of them). Clearly
describe the steps of your algorithm in your report.

Run the algorithm on the development sets RU/dev.in and ES/dev.in to generate 2-nd best
and 8-th best outputs. Write the outputs to RU/dev.p3.2nd.out, RU/dev.p3.8th.out and
ES/dev.p3.2nd.out, ES/dev.p3.8th.out. Report the precision, recall and F scores for the
outputs for both languages.

Hint: find the top-k best sequences using dynamic programming by modifying the original Viterbi
algorithm.

## Approach
### Revamped Viterbi Implementation:
The Viterbi algorithm is extended to maintain a list of candidate sequences for each state at each step.
- This list is sorted based on probabilities, and the top-k sequences are kept.
- The algorithm explores multiple sequences in parallel, allowing for the possibility of obtaining the k-th best sequence.

The revamped Viterbi implementation's modification allows it to generate and evaluate multiple sequence candidates, making it suitable for obtaining the k-th best sequence. This is why the revamped approach can achieve the desired k-th best output while the first implementation is designed for finding the single best output sequence.

In [11]:
from collections import defaultdict
import numpy as np

# get 2 lists, one for observations one for labels from training data
def read_observations_labels(input_file):
    # Initialize lists to store processed data
    all_observations = []
    all_labels = []
    
    # Open the input file for reading
    with open(input_file, 'r') as f:
        # Initialize temporary lists for the current group of observations and labels
        current_observation_group = []
        current_label_group = []
        
        # Iterate over each line in the file
        for line in f:
            # Check if the line is empty or contains only whitespace
            if line.isspace():
                # End of group: Append current observations and labels to the main lists
                all_observations.append(current_observation_group[:])
                all_labels.append(current_label_group[:])
                
                # Clear temporary lists to prepare for the next group
                current_observation_group.clear()
                current_label_group.clear()
            else:
                # Extract observation and label from the line
                observation, label = line.strip().rsplit(maxsplit=1)
                
                # Add the extracted observation and label to their respective lists
                current_observation_group.append(observation)
                current_label_group.append(label)
    
    # Return the lists of processed observations and labels
    return all_observations, all_labels


def revamped_transition_estimator(data):
    # Initialize a dictionary of dictionaries to store transition parameters
    trans_params = defaultdict(lambda: defaultdict(int))
    # Initialize count for the 'START' and 'STOP' transitions
    trans_params['START']['COUNT'] = 0
    trans_params['STOP']['COUNT'] = 0
    
    # Iterate through each sequence of labels in the input data
    for seq in data:
        prev_label = 'START'  # Initialize previous label as 'START'
        for curr_label in seq:
            # Count the transition from prev_label to curr_label
            trans_params[prev_label][curr_label] += 1
            # Increment the total count for transitions from prev_label
            trans_params[prev_label]['COUNT'] += 1
            prev_label = curr_label  # Update prev_label to current label
        
        # Count the transition to 'STOP' at the end of the sequence
        trans_params[prev_label]['STOP'] += 1
        # Increment the total count for transitions from prev_label
        trans_params[prev_label]['COUNT'] += 1
    
    # Calculate log-probabilities for the transitions
    log_probs = {}
    for prev_label, trans_counts in trans_params.items():
        log_probs[prev_label] = {}
        total_trans = trans_counts['COUNT']  # Total count of transitions from prev_label
        for curr_label, count in trans_counts.items():
            if curr_label != 'COUNT':  # Skip the 'COUNT' key
                # Calculate the log-probability for the transition
                log_probs[prev_label][curr_label] = np.log(count / total_trans)
    
    return log_probs


def revamped_emission_parameters(observations, labels, k=1):
    # Dictionary to store emission parameters for each label
    emission_params = {}
    # Dictionary to store vocabulary words
    transformed_vocab = {}
    
    # Loop through each sequence of labels and corresponding observations
    for lbl_idx in range(len(labels)):
        for obs_idx in range(len(labels[lbl_idx])):
            # Get the current label and observation
            current_label = labels[lbl_idx][obs_idx]
            current_observation = observations[lbl_idx][obs_idx]
            
            # If the current label is not in emission_params, initialize it
            if current_label not in emission_params:
                emission_params[current_label] = {'COUNT': k, '#UNK#': k}
            
            # Get the current count of the observation for the current label
            curr_count = emission_params[current_label].get(current_observation, 0)
            # Increment the count of the current observation for the current label
            emission_params[current_label][current_observation] = curr_count + 1
            # Increment the total count for the current label
            emission_params[current_label]['COUNT'] += 1
            # Add the current observation to the vocabulary
            transformed_vocab[current_observation] = True

    # Calculate log-probabilities for each emission parameter
    for lbl in emission_params.keys():
        for obs in emission_params[lbl].keys():
            if obs != 'COUNT':
                # Calculate and store the log-probability of the emission parameter
                emission_params[lbl][obs] = np.log(emission_params[lbl][obs] / emission_params[lbl]['COUNT'])
                
    return emission_params, transformed_vocab


def kth_best_viterbi(word_dict, sentence, trans_params, emiss_params, k_val):
    # Prepare the list of labels excluding 'START' and 'STOP'
    label_list = list(trans_params.keys())
    label_list.remove('START')
    label_list.remove('STOP')
    word_count = len(sentence)
    
    # Initialize the viterbi_table with the 'START' label and its score
    viterbi_table = []
    viterbi_table.append({'START': [(0, [])]})

    # Loop through each word in the sentence
    for i in range(word_count):
        viterbi_table.append({})  # Create a new row for the current word
        if word_dict.get(sentence[i], False):
            current_obs = sentence[i]  # Use the word itself if it's in the dictionary
        else:
            current_obs = '#UNK#'  # Use '#UNK#' if the word is not in the dictionary

        # Loop through each possible next label
        for next_label in label_list:
            obs_score = emiss_params[next_label].get(current_obs, -float('inf'))  # Get emission score

            entries = []
            # Loop through each possible previous label
            for prev_label in trans_params.keys():
                prev_entries = viterbi_table[i].get(prev_label, [])  # Get previous entries in the table
                for v_score, path in prev_entries:
                    trans_score = trans_params.get(prev_label, {}).get(next_label, -float('inf'))  # Get transition score
                    new_score = v_score + obs_score + trans_score  # Calculate combined score
                    if new_score > -float('inf'):
                        new_path = path.copy()
                        new_path.append(prev_label)
                        entries.append((new_score, new_path))
            entries.sort(key=lambda x: x[0])  # Sort entries based on score
            while len(entries) > k_val:
                entries.pop(0)  # Keep only top-k entries
            if len(entries) > 0:
                viterbi_table[i + 1][next_label] = entries  # Store the top-k entries in the table

        # Handle the case where no label is assigned to the current word
        if len(list(viterbi_table[i + 1].keys())) < 1:
            entries = []
            for prev_label in trans_params.keys():
                prev_entries = viterbi_table[i].get(prev_label, [])
                for v_score, path in prev_entries:
                    new_path = path.copy()
                    new_path.append(prev_label)
                    entries.append((v_score, new_path))
            entries.sort(key=lambda x: x[0])
            while len(entries) > k_val:
                entries.pop(0)
            if len(entries) > 0:
                viterbi_table[i + 1]['O'] = entries

    # Calculate the entries for the 'STOP' label
    entries = []
    for prev_label in trans_params.keys():
        prev_entries = viterbi_table[word_count].get(prev_label, [])
        for v_score, path in prev_entries:
            stop_score = trans_params.get(prev_label, {}).get('STOP', -float('inf'))
            new_score = v_score + stop_score
            if new_score > -float('inf'):
                new_path = path.copy()
                new_path.append(prev_label)
                entries.append((new_score, new_path))
    entries.sort(key=lambda x: x[0])
    while len(entries) > k_val:
        entries.pop(0)
    viterbi_table.append({'STOP': entries})  # Store the entries for 'STOP' label
    
    # Retrieve the final sequence by selecting the path with the highest probability
    final_seq = viterbi_table[-1]['STOP'][0][1]
    final_seq.pop(0)  # Remove the initial 'START' label
    return final_seq

# run prediction and save result to file
def write_labels_to_file(emission_parameters, transition_parameters, vocab, k, input_file, output_file):
    observations = create_observation_sequences(input_file)
    with open(output_file, 'w') as f:
        for observation in observations:
            labels = kth_best_viterbi(vocab, observation, transition_parameters, emission_parameters, k)
            for i in range(len(observation)):
                f.write(observation[i] + ' ' + labels[i] + '\n')
            f.write('\n')

## Prediction for ES

In [12]:
observations, labels = read_observations_labels('ES/train.txt')
transition_parameters = revamped_transition_estimator(labels)
emission_parameters, vocab = revamped_emission_parameters(observations, labels)

#generate output for k = 2 and k = 8
k = [2, 8]
for _k in k:
    if _k == 2:
        filename = 'dev.p3.2nd.out'
    else:
        filename = f'dev.p3.{_k}th.out'
    write_labels_to_file(emission_parameters, transition_parameters, vocab, _k, 'ES/dev.in', 'ES/' + filename)

### Results for ES
#### k=2
#Entity in gold data: 229
#Entity in prediction: 438

#Correct Entity : 119
Entity  precision: 0.2717
Entity  recall: 0.5197
Entity  F: 0.3568

#Correct Sentiment : 67
Sentiment  precision: 0.1530
Sentiment  recall: 0.2926
Sentiment  F: 0.2009

#### k=8
#Entity in gold data: 229
#Entity in prediction: 485

#Correct Entity : 109
Entity  precision: 0.2247
Entity  recall: 0.4760
Entity  F: 0.3053

#Correct Sentiment : 60
Sentiment  precision: 0.1237
Sentiment  recall: 0.2620
Sentiment  F: 0.1681

## Prediction for RU

In [13]:
observations, labels = read_observations_labels('RU/train.txt')
transition_parameters = revamped_transition_estimator(labels)
emission_parameters, vocab = revamped_emission_parameters(observations, labels)

#generate output for k = 2 and k = 8
k = [2, 8]
for _k in k:
    if _k == 2:
        filename = 'dev.p3.2nd.out'
    else:
        filename = f'dev.p3.{_k}th.out'
    write_labels_to_file(emission_parameters, transition_parameters, vocab, _k, 'RU/dev.in', 'RU/' + filename)

### Results for RU
#### k=2
#Entity in gold data: 389
#Entity in prediction: 700

#Correct Entity : 203
Entity  precision: 0.2900
Entity  recall: 0.5219
Entity  F: 0.3728

#Correct Sentiment : 126
Sentiment  precision: 0.1800
Sentiment  recall: 0.3239
Sentiment  F: 0.2314

#### k=8
#Entity in gold data: 389
#Entity in prediction: 801

#Correct Entity : 188
Entity  precision: 0.2347
Entity  recall: 0.4833
Entity  F: 0.3160

#Correct Sentiment : 102
Sentiment  precision: 0.1273
Sentiment  recall: 0.2622
Sentiment  F: 0.1714

# Part 4: Design Challenge PART 1
For our design, we propose the use of Higher Order Hidden Markov Models (HOHMM). 
In a Higher Order HMM, emission pand transition probabilities are conditioned on the current state and the previous several states. This allows the model to capture longer-range dependencies in the sequence. We hypothesise that this could help capture more complex sentiment patterns that extend over multiple words.

## 4.1 Data Processing
Here, we aim to use integers to index each possible state in order. This is done so as to enable the use of numpy arrays for calculating of Higher-order Transition Probabilities, that take into account not just the previous state but one before it that will be done in section `4.2`. 

In [14]:
import numpy as np
import pandas as pd

In [15]:
# Functions for this step 4.1
def read_data_from_file(file_path):
    """
    Seperates txt file into texts and labels arary
    """
    texts = []
    labels = []

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                text = parts[0]
                label = parts[1]
                texts.append(text)
                labels.append(label)
            elif len(parts) == 1:
                texts.append(line.strip())
    
    return texts, labels

def process_data_HOHMM(file_path):
    """
    Finds higher order transition probabilities
    INPUTS:
    - file_path (string): Path to the data file in txt format
    OUTPUTS:
    - texts: array containing all texts
    - labels: array containing coressponding labels of each text
    - states: array containing unique states
    - state_to_idx: mapping of states to a numerical index
    - label_idx: array of states in numerical idx, corresponding to texts
    
    
    """
    texts, labels = read_data_from_file(file_path)
    # Get states
    states = list(set(labels))
    
    # Define a mapping from states to indices
    state_to_idx = {state: idx for idx, state in enumerate(states)}
    
    # Get array of state indices corresponding to each text in the texts array
    label_idx = [state_to_idx[label] for label in labels]

    return texts, labels, states, state_to_idx, label_idx

In [16]:
# Data Processing for EU and RS
ES_train = "ES/train.txt"
RU_train = "RU/train.txt"

ES_texts, ES_labels, ES_states, ES_state_mappings, ES_label_idx = process_data_HOHMM(ES_train)
RU_texts, RU_labels, RU_states, RU_state_mappings, RU_label_idx = process_data_HOHMM(RU_train)

## 4.2 Calculation of Higher Order Transition Probabilities
In addition to standard HMM initialization, we consider the higher order transition probabilities. These probabilities involve multiple previous states, capturing longer-range dependencies.

Simply put, we need to find the parameter $a_{u, v, q}$ which is the probability of state $q$ appearing after $u, v$ appears.
$$a_{u, v, q} = P(q|y, v) = \frac{Count(u, v, q)}{Count(u,v)}$$

whereby $Count(u, v, q)$ is the number of times `q` appears after `u, v` appears before it

In [17]:
def find_a_HOHMM(states, label_idx):
    """
    Finds a_uvq, HO transition probabilities
    INPUTS:
    - states (arr): array containing states in dataset
    - label_udx (arr): Array of Indexed Labels for each corresponding text
    OUTPUTS:
    - a_uvq (np.array): np.array containign transition probabilities
    """
    # Find number of states
    num_states = len(states)
    
    # Initiate a np array to store HO a_uvq
    transition_probs = np.zeros((num_states, num_states, num_states))
    # Finding Count(u, v, q)
    for i in range(2, len(label_idx)):
        prev_state_1 = label_idx[i - 2]
        prev_state_2 = label_idx[i - 1]
        current_state = label_idx[i]
        
        transition_probs[prev_state_1, prev_state_2, current_state] += 1
        
    # Finding HO transition params
    count_uv = np.nansum(transition_probs, axis=2, keepdims=True) + 1e-10
    a_uvq = transition_probs / count_uv
    
    # Handle NaNs, convert to 0 probability
    a_uvq = np.nan_to_num(a_uvq, nan=0.0)

    return a_uvq

In [18]:
# Finding HO transition probabiltiies for ES and RU
ES_a = find_a_HOHMM(ES_states, ES_label_idx)
RU_a = find_a_HOHMM(RU_states, RU_label_idx)

## 4.3 Calculating Emission Probabilities
Follows that of normal HMM
$$b_u(o) = \frac{Count(u \to o)}{Count(u)}$$

However, we modify the function to handle np arrays. In order to take into account possible unique words, we create a vocab mapping too to prevent IndexErrors.

In [19]:
import numpy as np
import pandas as pd

def calculate_emission_probs(texts, labels, states):
    """
    Calculates emission probabilities for a Higher Order Hidden Markov Model (HOHMM).

    INPUTS:
    - texts (list): List of observed words/tokens.
    - labels (list): List of corresponding sentiment labels.
    - states (list): List of all possible states (sentiment labels).

    OUTPUTS:
    - emission_probs (np.array): Emission probabilities matrix (state x word).
    - words_to_idx (dict): Mappings from word to idx, following emissions probability np array
    """
    num_states = len(states)
    num_words = len(set(texts))
    
    # Emission probabilities (word given state)
    emission_probs = np.zeros((num_states, num_words))
    
    # Count occurrences of words for each state
    state_word_counts = np.zeros((num_states, num_words))

    # Mapping dict
    words_to_idx = {}
    idx_to_words = {}
    
    # Process the dataset
    for i in range(len(texts)):
        state_idx = labels[i]  # Use label directly
        word = texts[i]
        word_idx = hash(word) % num_words  # Simplified hashing

        # Store mappings
        words_to_idx[word] = word_idx
        idx_to_words[word_idx] = word
        state_word_counts[state_idx, word_idx] += 1
    
    # Calculate emission probabilities
    for state_idx in range(num_states):
        total_count = np.sum(state_word_counts[state_idx])
        emission_probs[state_idx] = state_word_counts[state_idx] / total_count
    
    return emission_probs, words_to_idx

In [20]:
RU_b, RU_wordmapping = calculate_emission_probs(RU_texts, RU_label_idx, RU_states)
ES_b, ES_wordmapping = calculate_emission_probs(ES_texts, ES_label_idx, ES_states)

## 4.4 Start Probabilities

In [21]:
def calculate_start_probs(label_indices, num_states):
    """
    Calculates start probabilities for a Higher Order Hidden Markov Model (HOHMM).

    INPUTS:
    - label_indices (list): List of label indices corresponding to each observation.
    - num_states (int): Number of possible states (sentiment labels).

    OUTPUTS:
    - start_probs (np.array): Start probabilities vector for each state.
    """
    start_counts = np.zeros(num_states)
    
    # Count occurrences of each state as the starting state
    start_counts[label_indices[0]] += 1
    
    for i in range(1, len(label_indices)):
        if label_indices[i - 1] != label_indices[i]:
            start_counts[label_indices[i]] += 1
    
    # Calculate start probabilities
    total_starts = np.sum(start_counts)
    start_probs = start_counts / total_starts
    
    return start_probs

In [22]:
ES_start_probs = calculate_start_probs(ES_label_idx, len(ES_states))
RU_start_probs = calculate_start_probs(RU_label_idx, len(RU_states))

print("Start Probabilities for ES:", ES_start_probs)
print("Start Probabilities for RU:", RU_start_probs)

Start Probabilities for ES: [0.02094241 0.004363   0.11082024 0.33653287 0.01977894 0.46829552
 0.03926702]
Start Probabilities for RU: [0.0378497  0.00511977 0.08118486 0.33900165 0.0149936  0.45785336
 0.06399707]


## 4.5 Viterbi Algorithm for Best Sequences
The function belows is differnt from the first due to the need of handling np.arrays, as well as a different way of calculating forward scores. Normally,
$$\pi(j+1, u) = \max_v \{\pi(j, v) \times b_u(x_{j+1}) \times a_{v, u} \}$$
However, we need to take into account that we are not just using the score of the previous states, but that of 2 states ago as well. Hence,
$$\pi(j+1, q) = \max_{u, v} \{\pi(j-1, u) \times b_q(x_{j+1}) \times a_{u, v} \times a_{v, q} \}$$

In [23]:
import numpy as np

def viterbi_algorithm(observations, states, start_probs, transition_probs, emission_probs, word_mapping):
    """
    Runs the Viterbi algorithm to find the most likely sequence of hidden states.

    INPUTS:
    - observations (list): List of observed words/tokens.
    - states (list): List of all possible states (sentiment labels).
    - start_probs (np.array): Initial state probabilities.
    - transition_probs (np.array): Transition probabilities matrix (state x state x state).
    - emission_probs (np.array): Emission probabilities matrix (state x word).
    - word_mapping (dict): Mapping of words to index in emissions probabiltiy np array

    OUTPUTS:
    - best_path (list): List of the most likely sequence of hidden states.
    """
    num_states = len(states)
    num_observations = len(observations)
    print(num_states, num_observations)
    
    # Create a Viterbi matrix to store probabilities and backpointers
    viterbi_matrix = np.zeros((num_states, num_observations))
    backpointers = np.zeros((num_states, num_observations), dtype=int)
    
    # Initialize the first column of the Viterbi matrix using start probabilities
    viterbi_matrix[:, 0] = start_probs * emission_probs[:, word_mapping[observations[0]]]
    
    # Fill in the Viterbi matrix and backpointers
    for t in range(1, num_observations):
        if observations[t] in word_mapping:
            for s in range(num_states):
                max_prob = -np.inf
                max_prev_state = None
                for prev_state_1 in range(num_states):
                    for prev_state_2 in range(num_states):
                        prob = viterbi_matrix[prev_state_2, t - 1] * transition_probs[prev_state_1, prev_state_2, s] * emission_probs[s, word_mapping[observations[t]]]
                        if prob > max_prob:
                            max_prob = prob
                            max_prev_state = prev_state_2
                viterbi_matrix[s, t] = max_prob
                backpointers[s, t] = max_prev_state
        
    # Backtrack to find the best path
    best_path = [0] * num_observations
    best_path[-1] = np.argmax(viterbi_matrix[:, -1])
    for t in range(num_observations - 2, 1, -1):
        best_path[t] = backpointers[best_path[t + 1], t+1]
    
    return best_path

# Example usage
ES_states_indexed = [i for i in range(len(ES_states))]
RU_states_indexed = [i for i in range(len(RU_states))]
# best_path_ES = viterbi_algorithm(ES_texts, ES_states_indexed, ES_start_probs, ES_a, ES_b, ES_wordmapping)
# best_path_RU = viterbi_algorithm(RU_texts, RU_states_indexed , RU_start_probs, RU_a, RU_b, RU_wordmapping)

## 4.6 Model Evaluation
Now that we've found our parameters $a_{u, v, q}, b_q(o)$, we can run it on the development, before saving the files to find their scores.

In [24]:
# Data Processing for EU and RS
ES_dev = "ES/dev.in"
RU_dev = "RU/dev.in"

dES_texts, dES_labels, dES_states, dES_state_mappings, dES_label_idx = process_data_HOHMM(ES_dev)
dRU_texts, dRU_labels, dRU_states, dRU_state_mappings, dRU_label_idx = process_data_HOHMM(RU_dev)

# Running viterbi using learned probabilities
best_path_ES = viterbi_algorithm(dES_texts, ES_states_indexed, ES_start_probs, ES_a, ES_b, ES_wordmapping)
best_path_RU = viterbi_algorithm(dRU_texts, RU_states_indexed , RU_start_probs, RU_a, RU_b, RU_wordmapping)

7 4312
7 6589


In [25]:
RU_state_mappings

{'B-neutral': 0,
 'I-neutral': 1,
 'B-negative': 2,
 'B-positive': 3,
 'I-negative': 4,
 'O': 5,
 'I-positive': 6}

In [26]:
# convert back to BIO labels
def convert_labels(best_path, state_mappings):
    for i in range(len(best_path)):
        best_path[i] = state_mappings[best_path[i]]

In [27]:
# Reverse mapping of idx to states 
r_map = {}
for k, v in RU_state_mappings.items():
    r_map[v] = k
convert_labels(best_path_ES, r_map)
convert_labels(best_path_RU, r_map)

In [28]:
# saving the files
write_sequences_to_file("ES/dev.p4.out", ES_texts, best_path_ES)
write_sequences_to_file("RU/dev.p4.out", RU_texts, best_path_RU)

### RU
#Entity in gold data: 389
#Entity in prediction: 434

#Correct Entity : 68
Entity  precision: 0.1567
Entity  recall: 0.1748
Entity  F: 0.1652

#Correct Sentiment : 0
Sentiment  precision: 0.0000
Sentiment  recall: 0.0000
Sentiment  F: 0.0000


### ES
#Entity in gold data: 229
#Entity in prediction: 252

#Correct Entity : 18
Entity  precision: 0.0714
Entity  recall: 0.0786
Entity  F: 0.0748

#Correct Sentiment : 0
Sentiment  precision: 0.0000
Sentiment  recall: 0.0000
Sentiment  F: 0.0000


Unfortunately, as seen from the test results, it may appear that a HOHMM might not be underfitting in our context as the training set might be too small to optimally calculate robust higher order transmission probabilities. Sequences of 3 states in a row leads to $7^3 = 343$ possible combinations, up from the $7^2 = 49$ combinations that having a 1-order HMM would entail. This leads us to believe that our model is severely underfit, and that the 343 possbilities of transmission probabilties $a_{u, v, q}$ of which might be hard to cover given the small number of training data.

# 5.0 Design Challenge PART 2: Improvements to HOHMM
As our team was dissatisfied with the results of HOHMM, we attempt another strategy in an attempt to improve upon the HOHMM model. In this part of the Design Challenge, we focus on using Laplace Smoothing.

Laplace Smoothing involves adding a small constant (usually denoted as $ε$ or $k$) to the observed counts before calculating probabilities. This ensures that even events that haven't been observed in the training data are assigned nonzero probabilities.

## 5.1 Benefits of Laplace Smoothing
1. Regularization of State Transitions:
HOHMMs can have complex state transition structures. Since the model is underfitting, it might not be capturing the true relationships between states. By applying Laplace smoothing to transition probabilities, we ensure that even less frequent or unobserved transitions receive nonzero probabilities. This can help the model learn more accurate transition patterns and improve its performance.
2. Dealing with Sparsity:
In complex models like HOHMMs, some state transitions or hidden state emissions might have low counts or be completely unobserved in the training data. Laplace smoothing can address this sparsity issue by adding a small constant to the counts, preventing zero probabilities. This is particularly important in underfitting scenarios where the model might not be generalizing well to unseen or rare events.
3. Improved Generalization:
Underfitting often leads to poor generalization, where the model struggles to make accurate predictions on unseen data. Laplace smoothing encourages a more balanced distribution of probabilities, helping the model generalize better by assigning nonzero probabilities to a wider range of events and transitions.
4. Balancing Model Complexity:
Underfitting can sometimes occur when the model's complexity doesn't match the complexity of the data. Laplace smoothing can act as a form of regularization, balancing the model's complexity and promoting a smoother distribution of probabilities, which can help with more accurate modeling.

## 5.2 Laplace Smoothing for Transition and Emission Probabilities
The smoothed transition probability from state $u, v$ to state $q$ is calculated as:
$$\text{Smoothed Transition Probability}(u, v \to q) = \frac{Count(u, v, q) + \epsilon}{Count(u, v) + N_t \times \epsilon}$$
wherbeby $N_t$ is the number of states.
The smoothed emission probability of observing emission $b$ in state $q$ is calculated as:
$$\text{Smoothed Emission Probability}(q \to o) = \frac{Count(q  \to o) + \epsilon}{Count(q) + N_e \times \epsilon}$$
wherbeby $N_e$ is the number of unique emissions.

Hence, we modify the previous HOHMM functions to include $\epsilon$, as well as take into account $N_e$ and $N_t$

In [29]:
def calculate_emission_probs_laplace(texts, labels, states, epsilon):
    """
    Calculates emission probabilities for a Higher Order Hidden Markov Model (HOHMM) + Laplace Smoothing

    INPUTS:
    - texts (list): List of observed words/tokens.
    - labels (list): List of corresponding sentiment labels.
    - states (list): List of all possible states (sentiment labels).
    - epsilon (float): Smoothing constant

    OUTPUTS:
    - emission_probs (np.array): Emission probabilities matrix (state x word).
    - words_to_idx (dict): Mappings from word to idx, following emissions probability np array
    """
    num_states = len(states)
    num_words = len(set(texts))
    
    # Emission probabilities (word given state)
    emission_probs = np.zeros((num_states, num_words))
    
    # Count occurrences of words for each state
    state_word_counts = np.zeros((num_states, num_words))

    # Mapping dict
    words_to_idx = {}
    idx_to_words = {}
    
    # Process the dataset
    for i in range(len(texts)):
        state_idx = labels[i]  # Use label directly
        word = texts[i]
        word_idx = hash(word) % num_words  # Simplified hashing

        words_to_idx[word] = word_idx
        idx_to_words[word_idx] = word
        state_word_counts[state_idx, word_idx] += 1
    
    # Calculate emission probabilities
    for state_idx in range(num_states):
        total_count = np.sum(state_word_counts[state_idx]) + num_words * epsilon # added epsilon to denom
        emission_probs[state_idx] = (state_word_counts[state_idx] + epsilon) / total_count # added epsilon to numerator
    
    return emission_probs, words_to_idx


def find_a_HOHMM_laplace(states, label_idx, epsilon):
    """
    Finds a_uvq, HO transition probabilities
    INPUTS:
    - states (arr): array containing states in dataset
    - label_udx (arr): Array of Indexed Labels for each corresponding text
    - epsilon (float): Smoothing constant
    
    OUTPUTS:
    - a_uvq (np.array): np.array containign transition probabilities
    """
    # Find number of states
    num_states = len(states)
    
    # Initiate a np array to store HO a_uvq
    transition_probs = np.zeros((num_states, num_states, num_states))
    # Finding Count(u, v, q)
    for i in range(2, len(label_idx)):
        prev_state_1 = label_idx[i - 2]
        prev_state_2 = label_idx[i - 1]
        current_state = label_idx[i]
        
        transition_probs[prev_state_1, prev_state_2, current_state] += 1
        
    # Finding HO transition params
    count_uv = np.nansum(transition_probs, axis=2, keepdims=True) + num_states * epsilon
    a_uvq = (transition_probs + epsilon) / count_uv
    
    # Handle NaNs, convert to 0 probability
    a_uvq = np.nan_to_num(a_uvq, nan=0.0)

    return a_uvq

## 5.3 Retraining of Model
Once again using `train.txt`, we try finding the new emission and transition probabilities using our new functions

In [30]:
# Defining initial epsilon
epsilon = 1

# Data Processing for EU and RS
ES_train = "ES/train.txt"
RU_train = "RU/train.txt"

ES_texts, ES_labels, ES_states, ES_state_mappings, ES_label_idx = process_data_HOHMM(ES_train)
RU_texts, RU_labels, RU_states, RU_state_mappings, RU_label_idx = process_data_HOHMM(RU_train)

# Finding HO transition probabiltiies and emission for ES and RU, with smoothing
ES_a = find_a_HOHMM_laplace(ES_states, ES_label_idx, epsilon)
RU_a = find_a_HOHMM_laplace(RU_states, RU_label_idx, epsilon)

RU_b, RU_wordmapping = calculate_emission_probs_laplace(RU_texts, RU_label_idx, RU_states, epsilon)
ES_b, ES_wordmapping = calculate_emission_probs_laplace(ES_texts, ES_label_idx, ES_states, epsilon)

# Calculating Start Probabilities
ES_start_probs = calculate_start_probs(ES_label_idx, len(ES_states))
RU_start_probs = calculate_start_probs(RU_label_idx, len(RU_states))

## 5.4 Prediction and Evaluation
We rerun the higher order Viterbi algorithm here before valuating it using the same metrics as before.

In [31]:
# Data Processing for EU and RS
ES_dev = "ES/dev.in"
RU_dev = "RU/dev.in"

dES_texts, dES_labels, dES_states, dES_state_mappings, dES_label_idx = process_data_HOHMM(ES_dev)
dRU_texts, dRU_labels, dRU_states, dRU_state_mappings, dRU_label_idx = process_data_HOHMM(RU_dev)

# Running viterbi using learned probabilities
best_path_ES = viterbi_algorithm(dES_texts, ES_states_indexed, ES_start_probs, ES_a, ES_b, ES_wordmapping)
best_path_RU = viterbi_algorithm(dRU_texts, RU_states_indexed , RU_start_probs, RU_a, RU_b, RU_wordmapping)

7 4312
7 6589


In [32]:
# Reverse mapping of idx to states 
r_map = {}
for k, v in RU_state_mappings.items():
    r_map[v] = k
convert_labels(best_path_ES, r_map)
convert_labels(best_path_RU, r_map)

In [33]:
# saving the files
write_sequences_to_file("ES/dev.p4.out", ES_texts, best_path_ES)
write_sequences_to_file("RU/dev.p4.out", RU_texts, best_path_RU)

In [34]:
# saving the files
write_sequences_to_file("P4 Tests/ESdev.p4.out", ES_texts, best_path_ES)
write_sequences_to_file("P4 Tests/RUdev.p4.out", RU_texts, best_path_RU)

## RU

#Entity in gold data: 389
#Entity in prediction: 431

#Correct Entity : 68
Entity  precision: 0.1578
Entity  recall: 0.1748
Entity  F: 0.1659

#Correct Sentiment : 0
Sentiment  precision: 0.0000
Sentiment  recall: 0.0000
Sentiment  F: 0.0000

## ES
#Entity in gold data: 229
#Entity in prediction: 248

#Correct Entity : 17
Entity  precision: 0.0685
Entity  recall: 0.0742
Entity  F: 0.0713

#Correct Sentiment : 0
Sentiment  precision: 0.0000
Sentiment  recall: 0.0000
Sentiment  F: 0.0000


## 5.5 Conclusion to HOHMM
There were insignificant changes to the HOHMM model, and we hereby conclude yet again that it was underfit. However, our team was still not satisfied and hence....,

# 6.0 Laplace Smoothing for the Initial HMM Model
As we saw some intial success with the earlier HMM model, we decided to use Laplace Smoothing to regularise our parameters in order to attain a better result. 

In Part 1, we were told to add a constant to the emission probability calculation to take into account unknown values from future inputs, and that serves as a means of improvement. As Laplace Smoothing essentially does the same thing but with more benefits as mentioned above, we believe that we could continue taking away lesson from our failed HOHMM model and apply it to improve our intitial model

## 6.1 Modifications of Functions
The smoothed transition probability from state $u$ to state $v$ is calculated as:
$$\text{Smoothed Transition Probability}(u \to v) = \frac{Count(u, v) + \epsilon}{Count(u) + N_t \times \epsilon}$$
wherbeby $N_t$ is the number of states.
The smoothed emission probability of observing emission $o$ in state $v$ is calculated as:
$$\text{Smoothed Emission Probability}(v \to o) = \frac{Count(v  \to o) + \epsilon}{Count(v) + N_e \times \epsilon}$$
wherbeby $N_e$ is the number of unique emissions.

In [35]:
def estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities
    train_words = []
    test_words = []
    
    # Split the file content into lines
    train_data_lines = training_file_content.strip().split("\n")
    test_data_lines = test_file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in train_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        if observation not in train_words:
            train_words.append(observation)
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}
    # Iterate through each line to extract observations from test set
    for line in test_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        else:
            observation = line
        if observation not in test_words:
            test_words.append(observation)

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: (count + epsilon) / (total_count + Ne * epsilon) for observation, count in observations_count.items()
        }



    return emission_params, list(set(test_words).union(train_words))
# Modified Transmission Parameters with Laplace

def estimate_transition_parameters_laplace(file_content, epsilon):
    transition_counts = {}
    transition_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Nt number of labels
    Nt = 7
    # Iterate through each line and extract observations and tags
    for line_index in range(len(lines)):
        next_line = ""
        curr_line = lines[line_index]
        if line_index < len(lines) - 1:
            next_line = lines[line_index + 1]
        
        if len(curr_line) == 0:
            continue
        if curr_line[-1] == 'O':
            curr_observation, curr_tag = curr_line[:-1], curr_line[-1]
        elif ('B-positive' in curr_line 
            or 'I-positive' in curr_line 
            or 'B-negative' in curr_line 
            or 'I-negative' in curr_line):
            curr_observation, curr_tag = curr_line[:-10], curr_line[-10:]
        elif ('B-neutral' in curr_line
            or 'I-neutral' in curr_line):
            curr_observation, curr_tag = curr_line[:-9], curr_line[-9:]
        
        if len(next_line) == 0: #true if line_index = len(lines - 1) or line_index is last index of a document
            pass
        elif next_line[-1] == 'O':
            next_observation, next_tag = next_line[:-1], next_line[-1]
        elif ('B-positive' in next_line 
            or 'I-positive' in next_line 
            or 'B-negative' in next_line 
            or 'I-negative' in next_line):
            next_observation, next_tag = next_line[:-10], next_line[-10:]
        elif ('B-neutral' in next_line
            or 'I-neutral' in next_line):
            next_observation, next_tag = next_line[:-9], next_line[-9:]        
        
        # Handle transitions from START
        if line_index == 0 or lines[line_index - 1] == "":
            if 'START' in transition_counts:
                transition_counts['START'][curr_tag] = transition_counts['START'].get(curr_tag, 0) + 1
            else:
                transition_counts['START'] = {curr_tag: 1}
                
        # Handle transitions to STOP
        if len(next_line) == 0:
            if curr_tag in transition_counts:
                transition_counts[curr_tag]['STOP'] = transition_counts[curr_tag].get('STOP', 0) + 1
            else:
                transition_counts[curr_tag] = {'STOP': 1}
                
        # Rest of the transitions
        else:
            if curr_tag in transition_counts:
                transition_counts[curr_tag][next_tag] = transition_counts[curr_tag].get(next_tag, 0) + 1
            else:
                transition_counts[curr_tag] = {next_tag: 1}
        

    # Calculate transition probabilities 
    for tag, next_tag_counts in transition_counts.items():
        total_count = sum(next_tag_counts.values()) 
        transition_params[tag] = {
            next_tag: (count + epsilon) / (total_count + Nt * epsilon) for next_tag, count in next_tag_counts.items()
        }

    return transition_params


## 6.2 Training the Initial Model with Laplace
Leveraging upon functions we wrote too for HOHMM in order to calculate `Ne`

In [36]:
# Data Processing for EU and RS
ES_train = "ES/train.txt"
RU_train = "RU/train.txt"

ES_texts, _, _, _, _ = process_data_HOHMM(ES_train)
RU_texts, _, _, _, _ = process_data_HOHMM(RU_train)

In [37]:
epsilon = 1.0

#ES
training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
Ne = len(set(ES_texts))
emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("P4 Tests/ESdev.p4.out", observation_sequences, best_sequences)

In [38]:
# RU
training_file_content = read_file("RU/train.txt")
test_file_content = read_file("RU/dev.in")
Ne = len(set(RU_texts))
emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
observation_sequences = create_observation_sequences("RU/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("P4 Tests/RUdev.p4.out", observation_sequences, best_sequences)

## 6.3 Evaluation of using Laplace on the Initial HMM

### ES
#Entity in gold data: 229
#Entity in prediction: 75

#Correct Entity : 67
Entity  precision: 0.8933
Entity  recall: 0.2926
Entity  F: 0.4408

#Correct Sentiment : 55
Sentiment  precision: 0.7333
Sentiment  recall: 0.2402
Sentiment  F: 0.3618

### RU
python3 evalResult.py RUdev.out RUdev.p4.out 

#Entity in gold data: 389
#Entity in prediction: 3867

#Correct Entity : 208
Entity  precision: 0.0538
Entity  recall: 0.5347
Entity  F: 0.0977

#Correct Sentiment : 150
Sentiment  precision: 0.0388
Sentiment  recall: 0.3856
Sentiment  F: 0.0705

As we can see, there are significant improvements to the F score of the ES set by using Laplace Smoothing. We hencefourth stick to the use of Smoothing Laplacing for it, incresaing the epsilon parameter further to try and improve our robustness on the developments set till we see no further significant improvements.

However, the RU set seems to experience a huge decrease in Fscore, and hence we don't use Laplace Smoothing for it.

## 6.4 Improving the $\epsilon$ Parameter

In [39]:
for epsilon in range(2, 20, 2):
    # ES
    training_file_content = read_file("ES/train.txt")
    test_file_content = read_file("ES/dev.in")
    Ne = len(set(ES_texts))
    emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
    transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
    observation_sequences = create_observation_sequences("ES/dev.in")
    best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
    # print(best_sequences)
       
    write_sequences_to_file(f"P4 Tests/ESdev{epsilon}.p4.out", observation_sequences, best_sequences)
    """
    # RU
    training_file_content = read_file("RU/train.txt")
    test_file_content = read_file("RU/dev.in")
    Ne = len(set(RU_texts))
    emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
    transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
    observation_sequences = create_observation_sequences("RU/dev.in")
    best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
    # print(best_sequences)
       
    write_sequences_to_file(f"P4 Tests/RUdev{epsilon}.p4.out", observation_sequences, best_sequences)
    """

Here are the results:
#### $\epsilon = 2$

python3 evalResult.py ESdev.out ESdev2.p4.out

#Entity in gold data: 229
#Entity in prediction: 49

#Correct Entity : 42
Entity  precision: 0.8571
Entity  recall: 0.1834
Entity  F: 0.3022

#Correct Sentiment : 34
Sentiment  precision: 0.6939
Sentiment  recall: 0.14

#### $\epsilon = 4$

python3 evalResult.py ESdev.out ESdev2.p4.out

#Entity in gold data: 229
#Entity in prediction: 49

#Correct Entity : 42
Entity  precision: 0.8571
Entity  recall: 0.1834
Entity  F: 0.3022

#Correct Sentiment : 34
Sentiment  precision: 0.6939
Sentiment  recall: 0.1485
Sentiment  F: 0.2446


In [40]:
epsilon = 0.7

training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
Ne = len(set(ES_texts))
emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file(f"P4 Tests/ESdev{epsilon}.p4.out", observation_sequences, best_sequences)

python3 evalResult.py ESdev.out ESdev0.7.p4.out

#Entity in gold data: 229
#Entity in prediction: 77

#Correct Entity : 69
Entity  precision: 0.8961
Entity  recall: 0.3013
Entity  F: 0.4510

#Correct Sentiment : 57
Sentiment  precision: 0.7403
Sentiment  recall: 0.2489
Sentiment  F: 0.3725


In [41]:
epsilon = 0.4

training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
Ne = len(set(ES_texts))
emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file(f"P4 Tests/ESdev{epsilon}.p4.out", observation_sequences, best_sequences)

python3 evalResult.py ESdev.out ESdev0.4.p4.out

#Entity in gold data: 229
#Entity in prediction: 92

#Correct Entity : 82
Entity  precision: 0.8913
Entity  recall: 0.3581
Entity  F: 0.5109

#Correct Sentiment : 67
Sentiment  precision: 0.7283
Sentiment  recall: 0.2926
Sentiment  F: 0.4174


As there seems to be an overall loss in the ES set's performance using Laplace Smoothing, it could be that the initial model was already fitted enough to handle data. As such we keep it as it is

## 6.5 Saving the Final Models
We save the develpoment results here from our final model choices

In [42]:
# ES with Epsilon = 1
epsilon = 1
training_file_content = read_file("RU/train.txt")
test_file_content = read_file("RU/dev.in")
Ne = len(set(RU_texts))
emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
observation_sequences = create_observation_sequences("RU/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("RU/dev.p4.out", observation_sequences, best_sequences)

In [43]:
# Initial RU
training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("ES/dev.p4.out", observation_sequences, best_sequences)

## 6.6. Running on Test

In [44]:
# RU with Epsilon = 1
epsilon = 1.0

#ES
training_file_content = read_file("ES/train.txt")
test_file_content = read_file("Test/ES/test.in")
Ne = len(set(ES_texts))
emission_params = estimate_emission_parameters_laplace(training_file_content, test_file_content, epsilon, Ne)[0]
transition_params = estimate_transition_parameters_laplace(training_file_content, epsilon)
observation_sequences = create_observation_sequences("Test/ES/test.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("P4 Tests/EStest.p4.out", observation_sequences, best_sequences)
write_sequences_to_file("ES/test.p4.out", observation_sequences, best_sequences)

In [45]:
# Initial ES
training_file_content = read_file("RU/train.txt")
test_file_content = read_file("Test/RU/test.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("Test/RU/test.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)

write_sequences_to_file("P4 Tests/RUtest.p4.out", observation_sequences, best_sequences)
write_sequences_to_file("RU/test.p4.out", observation_sequences, best_sequences)

Thanks for joining us on this arduous process of model improvement! :)